# Music Store Data Analysis Overview
This notebook demonstrates a step-by-step analysis of a music store's sales data to answer key business questions. The workflow includes:

1. **Data Ingestion**: Loading CSV files containing customer, invoice, invoice line, genre, and track data into pandas DataFrames.
2. **Data Cleaning**: Standardizing all column names to snake_case and ensuring consistent naming for ID fields (e.g., `customer_id`).
3. **Data Integration**: Merging tables to create a unified view for analysis, resolving column name conflicts (e.g., both genre and track have a `name` column).
4. **Business Insights**: Calculating metrics such as:
   - Country with the most customers
   - Top-spending customer
   - Revenue by music genre
   - Average transaction value per customer
   - Total revenue per year

Each step is documented and the code is modular for clarity and reproducibility.

In [9]:
import pandas as pd
import os

# Set data directory
DATA_DIR = 'music store data'

# Load CSV files
customer_df = pd.read_csv(os.path.join(DATA_DIR, 'customer.csv'))
invoice_df = pd.read_csv(os.path.join(DATA_DIR, 'invoice.csv'))
invoice_line_df = pd.read_csv(os.path.join(DATA_DIR, 'invoice_line.csv'))
genre_df = pd.read_csv(os.path.join(DATA_DIR, 'genre.csv'))
track_df = pd.read_csv(os.path.join(DATA_DIR, 'track.csv'))

## 1. Data Ingestion
We begin by importing necessary libraries and loading all relevant CSV files into pandas DataFrames. This step ensures we have access to all the data needed for analysis.

In [10]:
# Convert all column names to snake_case for each DataFrame
def to_snake_case(df):
    df.columns = [col.strip().replace(' ', '_').replace('-', '_').lower() for col in df.columns]
    return df

customer_df = to_snake_case(customer_df)
invoice_df = to_snake_case(invoice_df)
invoice_line_df = to_snake_case(invoice_line_df)
genre_df = to_snake_case(genre_df)
track_df = to_snake_case(track_df)

## 2. Data Cleaning
We standardize all column names to snake_case and ensure consistent naming for ID fields (e.g., `customer_id`). This makes the data easier to work with and reduces errors in later analysis steps.

In [11]:
# Further standardize column names: replace 'id' with '_id' where appropriate
def fix_id_columns(df):
    df.columns = [col.replace('id', '_id') if col.endswith('id') and not col.endswith('_id') else col for col in df.columns]
    return df

customer_df = fix_id_columns(customer_df)
invoice_df = fix_id_columns(invoice_df)
invoice_line_df = fix_id_columns(invoice_line_df)
genre_df = fix_id_columns(genre_df)
track_df = fix_id_columns(track_df)

## 3. Data Integration
We merge the tables to create a unified view for analysis. Special care is taken to resolve column name conflicts, such as when both genre and track tables have a `name` column.

In [12]:
# 1. Country with the most customers
country_counts = customer_df['country'].value_counts()
top_country = country_counts.idxmax()
top_country_count = country_counts.max()
print('Country with most customers:', top_country, f'({top_country_count})')

Country with most customers: USA (13)


## 4. Business Insights
We answer key business questions using the cleaned and integrated data. Each code cell below calculates and displays a specific metric, such as top customer, revenue by genre, and yearly revenue.

In [13]:
# 2. Customer who spent the most money
customer_invoice = pd.merge(invoice_df, customer_df, left_on='customer_id', right_on='customer_id')
customer_spending = customer_invoice.groupby(['customer_id', 'first_name', 'last_name'])['total'].sum().reset_index()
top_customer = customer_spending.sort_values('total', ascending=False).iloc[0]
print('Top customer:', top_customer['first_name'], top_customer['last_name'], f'- ${top_customer["total"]:.2f}')

Top customer: František Wichterlová - $144.54


In [14]:
# 3. Revenue by music genre
invoice_line_track = pd.merge(
    invoice_line_df, track_df, left_on='track_id', right_on='track_id', suffixes=('', '_track')
)
invoice_line_track_genre = pd.merge(
    invoice_line_track, genre_df, left_on='genre_id', right_on='genre_id', suffixes=('', '_genre')
)
# Use 'name_genre' for genre name
genre_revenue = invoice_line_track_genre.groupby('name_genre')['unit_price'].sum().reset_index().sort_values('unit_price', ascending=False)
print('\nRevenue by Genre:')
print(genre_revenue)


Revenue by Genre:
            name_genre  unit_price
15                Rock     2608.65
11               Metal      612.81
1   Alternative & Punk      487.08
10               Latin      165.33
13            R&B/Soul      157.41
2                Blues      122.76
9                 Jazz      119.79
0          Alternative      115.83
5       Easy Listening       73.26
12                 Pop       62.37
6    Electronica/Dance       54.45
3            Classical       46.53
14              Reggae       34.65
8          Hip Hop/Rap       32.67
7          Heavy Metal        7.92
16          Soundtrack        4.95
17            TV Shows        1.98
4                Drama        0.99


In [15]:
# 4. Average transaction value per customer
avg_transaction = customer_spending['total'].mean()
print('\nAverage transaction value per customer: $%.2f' % avg_transaction)


Average transaction value per customer: $79.82


In [16]:
# 5. Total revenue per year
invoice_df['invoice_date'] = pd.to_datetime(invoice_df['invoice_date'])
invoice_df['year'] = invoice_df['invoice_date'].dt.year
yearly_revenue = invoice_df.groupby('year')['total'].sum().reset_index()
print('\nTotal revenue per year:')
print(yearly_revenue)


Total revenue per year:
   year    total
0  2017  1201.86
1  2018  1147.41
2  2019  1221.66
3  2020  1138.50
